1. Es necesario utilizar unicamente los puntos sobre la esfera
que tengan todas sus puntos positivos. Esto porque sino
se obtienen rankings inversos.
2. No hay relacion entre el poset interseccion,
calculado usando los rankings de las puntuaciones,
con el __poset de puntos__
__poset de puntos__ poset que se obtiene al comparar
todas las entradas del vector que representa un nodo.
3. Es necesario mejorar la normalizacion
4. Los 1s del poset de puntos, siempre aparecen en el
poset borroso, calculado utilizando las puntuaciones.

In [ ]:
 import Distributions: MvNormal

In [1]:
using Posets

In [2]:
using Distributions
using Plots
using LinearAlgebra

using DelimitedFiles

In [3]:
using SparseArrays

# Poset

La función `generapuntuaciones_gaussian(a,b,c)` genera una tabla
de puntuaciones que corresponde a `a` rankings, con `b` nodos,
eligiendo los puntos en $\mathbb{R}^c$. También, devuelve como primer
argumento un poset calculado comparando todas las entradas de dos puntos.

La comparativa con la `energia_local` no es muy justa, ya que el poset
tiene pocas aristas, por lo que es más sencillo que los rankings
sean compatibles con él.

El corte $\alpha$ regresa una matriz que a mostrado el siguiente comportamiento:
los unos que aparecen en el __poset de puntos__ se encuentran siempre en la
del `poset_mn`.


In [20]:
posetdepuntos, puntuaciones = generapuntuaciones_gaussian(20,7,3);

In [21]:
ranks_univs = [sortperm(puntuaciones[:,x]#=, rev = true=#) for x in 1:size(puntuaciones)[2]];

In [22]:
[energia_local(r, posetdepuntos) for r in ranks_univs]

20-element Array{Int64,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [41]:
filter(x -> x>.5, matriz_union_rankings(ranks_univs, binario=false, promediado = true) )

7×7 Array{Float64,2}:
 0.0   0.0  0.0   0.95  0.0  0.0   0.85
 1.0   0.0  0.55  1.0   0.0  0.7   0.9
 0.8   0.0  0.0   1.0   0.0  0.8   1.0
 0.0   0.0  0.0   0.0   0.0  0.0   0.8
 0.75  0.0  0.65  1.0   0.0  0.75  1.0
 0.6   0.0  0.0   0.75  0.0  0.0   0.8
 0.0   0.0  0.0   0.0   0.0  0.0   0.0

In [32]:
poset_condercet = matriz_union_rankings(ranks_univs, binario=false, promediado = true) |> x -> αcut_poset(x, .5)

7×7 Array{Int64,2}:
 0  0  0  1  0  0  1
 1  0  1  1  0  1  1
 1  0  0  1  0  1  1
 0  0  0  0  0  0  1
 1  0  1  1  0  1  1
 1  0  0  1  0  0  1
 0  0  0  0  0  0  0

In [34]:
[energia_local(r, poset_condercet) for r in ranks_univs] |>countmap

Dict{Int64,Int64} with 3 entries:
  2 => 7
  3 => 3
  1 => 10

In [31]:
matriz_union_rankings(ranks_univs, binario=false, promediado = true) |> x -> αcut_poset(x, .5) |> SparseMatrixCSC|>isacyclic

true

In [25]:
?matriz_union_rankings

search: matriz_union_rankings



No documentation found.

`Posets.Generic.matriz_union_rankings` is a `Function`.

```
# 1 method for generic function "matriz_union_rankings":
[1] matriz_union_rankings(lista_ranks::Array{Array{T,1},1}; binario, promediado) where T<:Real in Posets.Generic at /home/david/.julia/dev/Posets/src/generic/Metodos.jl:353
```


In [23]:
posetdepuntos

7×7 Array{Int64,2}:
 0  0  0  0  0  0  0
 1  0  0  1  0  0  0
 0  0  0  0  0  0  1
 0  0  0  0  0  0  0
 0  0  0  1  0  0  1
 0  0  0  0  0  0  0
 0  0  0  0  0  0  0

In [6]:
poset_mn = puntuaciones |> fuzzy #|> transpose;

In [7]:
(posetdepuntos |> transpose).*poset_mn - (posetdepuntos |> transpose)

4×4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [10]:
posetdepuntos

4×4 Array{Int64,2}:
 0  0  0  0
 0  0  0  1
 1  0  0  1
 0  0  0  0

In [ ]:
posetdepuntos |> transpose

In [ ]:
matriz_interseccion_rankings(ranks_univs)

In [ ]:
posetdepuntos |> SparseMatrixCSC |> isacyclic

In [ ]:
fentropia(poset_mn)

In [ ]:
scatter([(puntosnodos[:,i][1],puntosnodos[:,i][2]) for i in 1:3]
   ,series_annotations = text.(1:3, :bottom))

## Condición

In [ ]:
listadiferencias = [0.]

In [ ]:
for _ in 1:10^2
posetdepuntos, puntuaciones = generapuntuaciones_gaussian(10,4,3);

ranks_univs = [sortperm(puntuaciones[:,x]#=, rev = true=#) for x in 1:size(puntuaciones)[2]];

poset_mn = puntuaciones |> fuzzy #|> transpose;

push!(listadiferencias, norm((posetdepuntos |> transpose).*poset_mn - (posetdepuntos |> transpose)) )
end

In [ ]:
listadiferencias |> countmap